In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader,TensorDataset
import torch.utils.data as data
import numpy as np
import scipy.io as sio
import math
import argparse
import random
import os
from My_Loss import HardTripletLoss
from tensorboardX import SummaryWriter

In [2]:
BATCH_SIZE = 64
EPISODE = 200000
TEST_EPISODE = 1000
LEARNING_RATE =5e-5
Weight_Deacy = 1e-5
GPU = 0
Margin = 3

In [3]:
print("init dataset")
##################################参数##################################################################
dataroot = '../data'
dataset = 'CUB1_data'
image_embedding = 'res101'               #ResNet101层
class_embedding = 'original_att'         #属性表达 85-d
#######################################读取视觉特征###################################################################

matcontent = sio.loadmat(dataroot + "/" + dataset + "/" + image_embedding + ".mat")  #scipy loadmat
 
feature = matcontent['features'].T         #转置 30478x2048 每一行是一个完整的样本

label = matcontent['labels'].astype(int).squeeze() - 1   #matlab begin 1 ,numpy begin 0
########################################读取属性特征###########################################################

matcontent = sio.loadmat(dataroot + "/" + dataset + "/" + class_embedding + "_splits.mat")

    
# numpy array index starts from 0, matlab starts from 1
trainval_loc = matcontent['trainval_loc'].squeeze() - 1    #squeeze()去掉维度中的1 AxBx1 --->AxB

test_seen_loc = matcontent['test_seen_loc'].squeeze() - 1
test_unseen_loc = matcontent['test_unseen_loc'].squeeze() - 1

attribute = matcontent['att'].T    #转置 50x85 每行是整个属性向量

x = feature[trainval_loc]                      # train_features trainval里面是图片的编号 begin with 0 ，19832个
train_label = label[trainval_loc].astype(int)  # train_label  int类型没变 每个图片的lable 19832个
train_id = np.unique(train_label)

att = attribute[train_label]                   # train attributes 每个图片的属性 19832个

########################add negative pairs#######################
#x_negative = np.empty_like(x)
#x_negative_label = np.empty_like(train_label)
#print(x.shape[0])

#for i in range(x.shape[0]):
#    pick=np.random.choice(np.where(train_label[i]!=train_id)[0], replace=True)
#    x_negative[i] = x[pick]
#    x_negative_label[i] = train_label[pick]
    
x_test = feature[test_unseen_loc]                   # test_feature 5685个
test_label = label[test_unseen_loc].astype(int)     # test_label   5685个

x_test_seen = feature[test_seen_loc]                #test_seen_feature 4958个
test_label_seen = label[test_seen_loc].astype(int)  # test_seen_label  4958个
    
test_id = np.unique(test_label)                     # test_id  10个类 ，unique去重
att_pro = attribute[test_id]                        # test_attribute 每一类的属性向量 10x85

# train set
train_features = torch.from_numpy(x)   #np-->tensor
#train_fearures_negative = torch.from_numpy(x_negative)

sample_attributes=[]
train_label = torch.from_numpy(train_label).unsqueeze(1) #每张图片的属性转化 ，unsqueeze(1)就是插入到第一维度 AxB维-->Ax1xB
#train_negative_label = torch.from_numpy(x_negative_label).unsqueeze(1)
# attributes
all_attributes = np.array(attribute)  #所有50类属性转变为numpy数组???属性向量仍然用的numpy类型 没有转化为pytorch
#print(all_attributes)
    
#print('-'*50)
attributes = torch.from_numpy(attribute) 
#print(attribute)
# test set

test_features = torch.from_numpy(x_test)
#print(test_features.shape)

test_label = torch.from_numpy(test_label).unsqueeze(1)
#print(test_label.shape)

testclasses_id = np.array(test_id)
#print(testclasses_id.shape)

test_attributes = torch.from_numpy(att_pro).float()
#print(test_attributes.shape)

test_seen_features = torch.from_numpy(x_test_seen)
#print(test_seen_features.shape)

test_seen_label = torch.from_numpy(test_label_seen)

train_data = TensorDataset( train_label, train_features )
#train_data = TensorDataset(train_label, train_features, train_fearures_negative)

#################here need new code to make triplet data#####################
print('-'*100)

init dataset
----------------------------------------------------------------------------------------------------


In [4]:
from my_net_3 import AttributeNetwork
from my_net_3 import MetricNetwork

In [5]:
# init network
print("init networks")
attribute_network = AttributeNetwork(312,1024,2048)  #85d属性 1024隐藏层 2048输出 85d到2048d
metric_network = MetricNetwork(2048,1024,2048)
#triplet_network = TripletNetwork(attribute_network, metric_network)  #metric learning   
attribute_network.cuda(GPU) 
metric_network.cuda(GPU)
#attribute_network_optim = torch.optim.Adam(attribute_network.parameters(), lr=LEARNING_RATE, weight_decay=1e-5)
#优化器adam 学习率 正则1e-5

#attribute_network_scheduler = StepLR(attribute_network_optim, step_size=200000, gamma=0.5)
#学习率每200k步 乘0.5
attribute_network_optim = torch.optim.Adam(attribute_network.parameters(), lr=LEARNING_RATE,weight_decay=Weight_Deacy)
metric_network_optim = torch.optim.Adam(metric_network.parameters(), lr=LEARNING_RATE,weight_decay=Weight_Deacy)
#
#triplet_network_optim = torch.optim.SGD(triplet_network.parameters(), lr=LEARNING_RATE,momentum=0.9 , 
#                                         weight_decay=Weight_Deacy)

attribute_network_scheduler = StepLR(attribute_network_optim , step_size=200000 , gamma=1)
metric_network_scheduler = StepLR(metric_network_optim , step_size=200000 , gamma=1)
#
print('-'*100)

init networks
----------------------------------------------------------------------------------------------------


In [6]:
def compute_accuracy(test_features, test_label, test_id, test_attributes):
    
    test_data = TensorDataset(test_features, test_label)
    test_batch = 32
    test_loader = DataLoader(test_data, batch_size=test_batch, shuffle=False)
    total_rewards = 0

    sample_labels = test_id
    sample_attributes = test_attributes
    class_num = sample_attributes.shape[0]
    test_size = test_features.shape[0]

    print("class num:", class_num)

    for batch_features,batch_labels in test_loader:

        batch_size = batch_labels.shape[0]
        batch_features_ext = torch.from_numpy(batch_features.numpy().repeat(class_num, 0))
        batch_features_ext = metric_network(Variable(batch_features_ext).cuda(GPU).float())  # 32*1024

        #print(batch_features_ext)

        sample_features = metric_network(attribute_network(Variable(sample_attributes).cuda(GPU).float()))
        sample_features_ext = sample_features.repeat(batch_size, 1)
        #print(sample_features_ext.shape)


        relations = F.pairwise_distance(batch_features_ext, sample_features_ext, 2).view(-1, class_num)
        re_batch_labels = []
        for label in batch_labels.numpy():
            index = np.argwhere(sample_labels == label)
            re_batch_labels.append(index[0][0])
        re_batch_labels = torch.cuda.LongTensor(re_batch_labels)


        _, predict_labels = torch.min(relations.data, 1)
        #print(predict_labels)
        rewards = [1 if predict_labels[j] == re_batch_labels[j] else 0 for j in range(batch_size)]
        total_rewards += np.sum(rewards)
    test_accuracy = total_rewards/1.0/test_size
    return  test_accuracy

In [7]:
def compute_accuracy_per_class(test_features, test_label, test_id, test_attributes):
    
    test_data = TensorDataset(test_features, test_label)
    test_batch = 32
    test_loader = DataLoader(test_data, batch_size=test_batch, shuffle=False)
    total_rewards = 0
    #print(test_features.size())
    sample_labels = test_id
    sample_attributes = test_attributes
    class_num = sample_attributes.shape[0]
    test_size = test_features.shape[0]
    per_class_num = np.zeros(class_num)
    per_class_true= np.zeros(class_num)

    print("class num:", class_num)

    for batch_features,batch_labels in test_loader:

        batch_size = batch_labels.shape[0]
        batch_features_ext = torch.from_numpy(batch_features.numpy().repeat(class_num, 0))
        batch_features_ext = metric_network(Variable(batch_features_ext).cuda(GPU).float())  # 32*1024


        sample_features = metric_network(attribute_network(Variable(sample_attributes).cuda(GPU).float()))
        sample_features_ext = sample_features.repeat(batch_size, 1)


        relations = F.pairwise_distance(batch_features_ext, sample_features_ext, 2).view(-1, class_num)
        re_batch_labels = []
        for label in batch_labels.numpy():
            index = np.argwhere(sample_labels == label)
            re_batch_labels.append(index[0][0])
        re_batch_labels_id, batch_per_num = np.unique(re_batch_labels , return_counts=True) 
        re_batch_labels = torch.cuda.LongTensor(re_batch_labels)
        
        for each in range(re_batch_labels_id.size):
            #print(re_batch_labels_id[each])
            #print(batch_per_num[each])
            per_class_num[re_batch_labels_id[each]] = per_class_num[re_batch_labels_id[each]] + batch_per_num[each]
        #print(re_batch_labels_id)
        #print('-'*100)
        #print(batch_per_num)
        #print('-'*100)


        _, predict_labels = torch.min(relations.data, 1)
        for j in range(batch_size):
            if predict_labels[j] == re_batch_labels[j]:
                per_class_true[re_batch_labels[j]] = per_class_true[re_batch_labels[j]] + 1
            
        
        
        rewards = [1 if predict_labels[j] == re_batch_labels[j] else 0 for j in range(batch_size)]
        total_rewards += np.sum(rewards)
    per_accuracy = per_class_true[np.nonzero(per_class_num)] / per_class_num[np.nonzero(per_class_num)]
    
    test_accuracy = np.sum(per_accuracy)/1.0/np.count_nonzero(per_class_num)
    test_accuracy2 = total_rewards/1.0/test_size
    return  test_accuracy , test_accuracy2

In [8]:
print("training...")
last_accuracy = 0.0
loss_zero_num = 0
#writer = SummaryWriter()
for episode in range(EPISODE):
    #attribute_network.train()
    attribute_network_scheduler.step(episode)
    metric_network_scheduler.step(episode)

    train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)

    batch_labels, batch_features = train_loader.__iter__().next()
    batch_id = np.unique(batch_labels)

    batch_attributes = torch.Tensor([all_attributes[i] for i in batch_id]).squeeze(1)
    batch_features_ext = torch.from_numpy(batch_features.numpy().repeat(batch_id.size, 0))
    batch_attributes_ext = batch_attributes.repeat(BATCH_SIZE, 1)

    batch_features_ext = Variable(batch_features_ext).cuda(GPU).float()  # 32*2048
    batch_attributes_ext = Variable(batch_attributes_ext).cuda(GPU)
    

    re_batch_labels = []
    for label in batch_labels.numpy():
        index = np.argwhere(batch_id == label)
        re_batch_labels.append(index[0][0])
    re_batch_labels = torch.cuda.LongTensor(re_batch_labels)
    re_batch_labels = Variable(re_batch_labels).cuda(GPU)
    

    criterion = HardTripletLoss(margin = Margin).cuda(GPU)
    triplet_loss= criterion(metric_network(attribute_network(batch_attributes_ext)), 
                            metric_network(batch_features_ext), re_batch_labels)
    if triplet_loss == 0:
        loss_zero_num = loss_zero_num + 1
    metric_network.zero_grad()
    attribute_network.zero_grad()
    
    triplet_loss.backward()
    
    attribute_network_optim.step()
    metric_network_optim.step()
    
    if (episode+1)%200 == 0 or episode==0:
        print("episode:", episode+1, "loss", triplet_loss)
        print('loss_zero_number= ',loss_zero_num)
        #writer.add_scalar('data/loss_zero_number', loss_zero_num, episode)
        loss_zero_num = 0
        #writer.add_scalar('data/loss', triplet_loss, episode)
    if (episode+1)%200 == 0 or episode==0:
        print("Testing...")
        #attribute_network.eval()
        zsl_accuracy ,zsl_accuracy2= compute_accuracy_per_class(test_features, test_label, test_id, test_attributes)
        gzsl_unseen_accuracy ,gzsl_unseen_accuracy2= compute_accuracy_per_class(test_features, test_label, np.arange(200), attributes)
        gzsl_seen_accuracy ,gzsl_seen_accuracy2 = compute_accuracy_per_class(test_seen_features, test_seen_label, np.arange(200), attributes)
        H = 2 * gzsl_seen_accuracy * gzsl_unseen_accuracy / (gzsl_unseen_accuracy + gzsl_seen_accuracy)
        H2 = 2 * gzsl_seen_accuracy2 * gzsl_unseen_accuracy2 / (gzsl_unseen_accuracy2 + gzsl_seen_accuracy2)
        print('zsl:', zsl_accuracy)
        #print('zsl:', zsl_accuracy2)
        print('gzsl: unseen=%.4f , seen=%.4f , h=%.4f' % (gzsl_unseen_accuracy , gzsl_seen_accuracy, H))
        #print('gzsl: unseen=%.4f , seen=%.4f , h=%.4f' % (gzsl_unseen_accuracy2 , gzsl_seen_accuracy2, H2))
        print('_'*100)
        #writer.add_scalar('data/zsl_accuracy', zsl_accuracy, episode)
        #writer.add_scalar('data/gzsl_unseen_accuracy', gzsl_unseen_accuracy, episode)
        #writer.add_scalar('data/gzsl_seen_accuracy', gzsl_seen_accuracy, episode)
        #writer.add_scalar('data/H', H, episode)
#writer.export_scalars_to_json("./test.json")
#writer.close()

training...
episode: 1 loss tensor(3.0593, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.024
gzsl: unseen=0.0000 , seen=0.0067 , h=0.0000
____________________________________________________________________________________________________
episode: 200 loss tensor(2.9691, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.05266666666666667
gzsl: unseen=0.0363 , seen=0.0450 , h=0.0402
____________________________________________________________________________________________________
episode: 400 loss tensor(2.9054, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.1206779661016949
gzsl: unseen=0.0583 , seen=0.0647 , h=0.0613
____________________________________________________________________________________________________
episode: 600 loss tensor(2.7456, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num: 200
cla

episode: 5600 loss tensor(2.0806, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.5313507459528036
gzsl: unseen=0.4257 , seen=0.5018 , h=0.4606
____________________________________________________________________________________________________
episode: 5800 loss tensor(1.9672, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.5305658295275589
gzsl: unseen=0.4332 , seen=0.4967 , h=0.4628
____________________________________________________________________________________________________
episode: 6000 loss tensor(1.8774, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.5361048217154157
gzsl: unseen=0.4355 , seen=0.4835 , h=0.4582
____________________________________________________________________________________________________
episode: 6200 loss tensor(1.9184, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num: 2

episode: 11200 loss tensor(0.7261, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.5216431773582805
gzsl: unseen=0.4067 , seen=0.5090 , h=0.4521
____________________________________________________________________________________________________
episode: 11400 loss tensor(1.3280, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.524628754693876
gzsl: unseen=0.4228 , seen=0.5048 , h=0.4602
____________________________________________________________________________________________________
episode: 11600 loss tensor(1.6419, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.5094864668812289
gzsl: unseen=0.4073 , seen=0.5110 , h=0.4533
____________________________________________________________________________________________________
episode: 11800 loss tensor(1.7934, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num

episode: 16800 loss tensor(0.9404, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.48873563922045554
gzsl: unseen=0.3681 , seen=0.5086 , h=0.4271
____________________________________________________________________________________________________
episode: 17000 loss tensor(0.4907, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.4936917697074529
gzsl: unseen=0.3666 , seen=0.5141 , h=0.4280
____________________________________________________________________________________________________
episode: 17200 loss tensor(0.8816, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.5201551804480601
gzsl: unseen=0.3992 , seen=0.5172 , h=0.4506
____________________________________________________________________________________________________
episode: 17400 loss tensor(0.4154, device='cuda:0')
loss_zero_number=  0
Testing...
class num: 50
class n

episode: 22400 loss tensor(0.2455, device='cuda:0')
loss_zero_number=  7
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.4969276107297898
gzsl: unseen=0.3799 , seen=0.5122 , h=0.4362
____________________________________________________________________________________________________
episode: 22600 loss tensor(0.4242, device='cuda:0')
loss_zero_number=  9
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.5068647283078732
gzsl: unseen=0.3843 , seen=0.5163 , h=0.4406
____________________________________________________________________________________________________
episode: 22800 loss tensor(0.4901, device='cuda:0')
loss_zero_number=  6
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.5057924361840277
gzsl: unseen=0.3764 , seen=0.5068 , h=0.4320
____________________________________________________________________________________________________
episode: 23000 loss tensor(0.4734, device='cuda:0')
loss_zero_number=  5
Testing...
class num: 50
class nu

...

episode: 172400 loss tensor(0., device='cuda:0')
loss_zero_number=  189
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.48111388998725546
gzsl: unseen=0.3731 , seen=0.5253 , h=0.4363
____________________________________________________________________________________________________
episode: 172600 loss tensor(0., device='cuda:0')
loss_zero_number=  183
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.456823718258738
gzsl: unseen=0.3364 , seen=0.5360 , h=0.4133
____________________________________________________________________________________________________
episode: 172800 loss tensor(0., device='cuda:0')
loss_zero_number=  189
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.4743191605188771
gzsl: unseen=0.3546 , seen=0.5209 , h=0.4219
____________________________________________________________________________________________________
episode: 173000 loss tensor(0., device='cuda:0')
loss_zero_number=  187
Testing...
class num: 50
class num: 2

episode: 178000 loss tensor(0., device='cuda:0')
loss_zero_number=  189
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.481202662125468
gzsl: unseen=0.3689 , seen=0.5203 , h=0.4317
____________________________________________________________________________________________________
episode: 178200 loss tensor(0., device='cuda:0')
loss_zero_number=  194
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.4842236421169926
gzsl: unseen=0.3657 , seen=0.5357 , h=0.4346
____________________________________________________________________________________________________
episode: 178400 loss tensor(0., device='cuda:0')
loss_zero_number=  189
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.4676699027958724
gzsl: unseen=0.3418 , seen=0.5344 , h=0.4170
____________________________________________________________________________________________________
episode: 178600 loss tensor(0., device='cuda:0')
loss_zero_number=  194
Testing...
class num: 50
class num: 20

episode: 183600 loss tensor(0., device='cuda:0')
loss_zero_number=  191
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.4626245541900802
gzsl: unseen=0.3419 , seen=0.5383 , h=0.4182
____________________________________________________________________________________________________
episode: 183800 loss tensor(0., device='cuda:0')
loss_zero_number=  195
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.4589422008010111
gzsl: unseen=0.3332 , seen=0.5382 , h=0.4116
____________________________________________________________________________________________________
episode: 184000 loss tensor(0., device='cuda:0')
loss_zero_number=  186
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.45070153654042355
gzsl: unseen=0.3281 , seen=0.5253 , h=0.4039
____________________________________________________________________________________________________
episode: 184200 loss tensor(0., device='cuda:0')
loss_zero_number=  188
Testing...
class num: 50
class num: 

episode: 189200 loss tensor(0., device='cuda:0')
loss_zero_number=  190
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.4818705060485427
gzsl: unseen=0.3605 , seen=0.5155 , h=0.4243
____________________________________________________________________________________________________
episode: 189400 loss tensor(0., device='cuda:0')
loss_zero_number=  190
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.4827122620767373
gzsl: unseen=0.3506 , seen=0.5235 , h=0.4199
____________________________________________________________________________________________________
episode: 189600 loss tensor(0., device='cuda:0')
loss_zero_number=  191
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.4724654454580924
gzsl: unseen=0.3492 , seen=0.5198 , h=0.4177
____________________________________________________________________________________________________
episode: 189800 loss tensor(0., device='cuda:0')
loss_zero_number=  189
Testing...
class num: 50
class num: 2

episode: 194800 loss tensor(0., device='cuda:0')
loss_zero_number=  189
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.47527376654310693
gzsl: unseen=0.3574 , seen=0.5200 , h=0.4236
____________________________________________________________________________________________________
episode: 195000 loss tensor(0., device='cuda:0')
loss_zero_number=  189
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.4670097633146969
gzsl: unseen=0.3535 , seen=0.5193 , h=0.4206
____________________________________________________________________________________________________
episode: 195200 loss tensor(0., device='cuda:0')
loss_zero_number=  193
Testing...
class num: 50
class num: 200
class num: 200
zsl: 0.4817945622716077
gzsl: unseen=0.3813 , seen=0.5212 , h=0.4404
____________________________________________________________________________________________________
episode: 195400 loss tensor(0., device='cuda:0')
loss_zero_number=  188
Testing...
class num: 50
class num: 